Recreation of ATL07 heights using ATL03 photons

NOTE: Computing uncorrected heights

Jesse Wimert

April 2024

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd

#
# Fine track routines
#
from fine_trap import fine_track
from fine_trap import plot_finetrack_seg

#
# For plotting ground tracks
#
from matplotlib.gridspec import GridSpec
import cartopy.crs as ccrs
import cartopy.feature as ft
import glob
import os
from pyproj import Proj, transform
from scipy import stats

In [ ]:
##
## See below for instructions for selecting data to process
##


In [ ]:
#
# Set input ATL03 and ATL07 files
#

ATL03_file = '/Users/jwimert/Desktop/961a5_test/05850201/ATL03_20190204180611_05850204_961a5_01.h5'
ATL07_file = '/Users/jwimert/Desktop/961a5_test/05850201/ATL07-01_20190204174514_05850201_961a5_02.h5'


#
# Set input waveform table
#
wf_table_file1 = '/Users/jwimert/Desktop/961a5_test/sea_ice_waveforms_gauss_020585_1.h5'
wf_table_file2 = '/Users/jwimert/Desktop/961a5_test/sea_ice_waveforms_gauss_020585_2.h5'

#
# Output directory
#
output_path = '/Users/jwimert/Desktop/jupyter_test/'

#
# Print debug plots?
#
print_debug = 'y'
# print_debug = 'n'


In [ ]:
#
#
###################Read ATL07 file###################################
# 
# 
# 
print('')
print('Reading ATL07 file...')
print(ATL07_file)
print('=====================================')

#
# Open file
#
hf = h5py.File(ATL07_file, 'r')

#
# Set processing to center strong beam
#
orientation = hf.get('/orbit_info/sc_orient')[0]

if orientation == 0:
    beam = 'gt2l'
elif orientation == 1:
    beam = 'gt2r'

tep_used = hf.get('/ancillary_data/fine_surface_finding/tep_used_gt2_strong')[0]

region = hf.get('/ancillary_data/start_region')[0]

#
# set subset of data to read (seg0 : seg1)
#
gtx = hf.get(beam)


ATL07_delta_time = gtx.get('sea_ice_segments/delta_time')[:]
ATL07_dist_x = gtx.get('sea_ice_segments/seg_dist_x')[:]
ATL07_lat = gtx.get('sea_ice_segments/latitude')[:]
ATL07_lon = gtx.get('sea_ice_segments/longitude')[:]

heightx = hf.get(beam+'/sea_ice_segments/heights')
ATL07_h_uncorr = heightx.get('height_segment_height_uncorr')[:]
ATL07_seg_length = heightx.get('height_segment_length_seg')[:]
ATL07_coarse_mn = gtx.get('sea_ice_segments/stats/height_coarse_mn')[:]

ATL07_h_corr = heightx.get('height_segment_height')[:]
ATL07_w_gauss = heightx.get('height_segment_w_gaussian')[:]
ATL07_htcorr_skew = heightx.get('height_segment_htcorr_skew')[:]
ATL07_rms = heightx.get('height_segment_rms')[:]

ATL07_n_pulse = heightx.get('height_segment_n_pulse_seg')[:]
ATL07_n_pulse_used = heightx.get('height_segment_n_pulse_seg_used')[:]
ATL07_quality_flag = heightx.get('height_segment_fit_quality_flag')[:]

ATL07_n_photons_actual = gtx.get('sea_ice_segments/stats/n_photons_actual')[:]
ATL07_n_photons_used = gtx.get('sea_ice_segments/stats/n_photons_used')[:]
ATL07_photon_rate = gtx.get('sea_ice_segments/stats/photon_rate')[:]

ATL07_fpb_corr = gtx.get('sea_ice_segments/stats/fpb_corr')[:]
ATL07_fpb_width = gtx.get('sea_ice_segments/stats/fpb_corr_width')[:]
ATL07_fpb_strength = gtx.get('sea_ice_segments/stats/fpb_strength')[:]
ATL07_fpb_avg_dt = gtx.get('sea_ice_segments/stats/fpb_avg_dt')[:]

# ATL07_histogram = gtx.get('sea_ice_segments/stats/hist_photon_heights')[:]

hf.close()

print(' ')
print('orientation:')
print(orientation)
print('beam:')
print(beam)
print('delta time')
print(np.min(ATL07_delta_time), np.max(ATL07_delta_time))
print('dist_x')
print(np.min(ATL07_dist_x), np.max(ATL07_dist_x))
print('TEP used')
print(tep_used)
print('region')
print(region)

In [ ]:
#
#
###################Read ATL03 file###################################
# 
# 
# 
print('')
print('Reading ATL03 file...')
print(ATL03_file)
print('=====================================')

#
# Open file
#
hf = h5py.File(ATL03_file, 'r')

#
# Read photon data data 
#
ATL03_ph_delta_time = hf[beam + '/heights/delta_time'][:]
ATL03_ph_height = hf[beam + '/heights/h_ph'][:]
ATL03_ph_conf = hf[beam + '/heights/signal_conf_ph'][:]
ATL03_ph_x = hf[beam + '/heights/dist_ph_along'][:]
#
# Pulse number needed to find specular returns, count n_shots for photon rate
#
ATL03_mframe = hf[beam + '/heights/pce_mframe_cnt'][:]
ATL03_pulse = hf[beam + '/heights/ph_id_pulse'][:]

#
# Read distance and photon count per geobin
#
ATL03_seg_dist_x = hf[beam + '/geolocation/segment_dist_x'][:]
ATL03_ph_index_beg = hf[beam + '/geolocation/ph_index_beg'][:]
ATL03_seg_ph_count = hf[beam + '/geolocation/segment_ph_cnt'][:]

#
# Read calibration data
# Only need dead_time_strong beam for this test
# CAL19 data identical across all beams, read from any
#

deadtime_read = hf['ancillary_data/calibrations/dead_time/' + beam + '/dead_time'][:]

dead_time_s = (np.sum(deadtime_read[0:16])/16.0)* 1.0e9
dead_time_w = (np.sum(deadtime_read[16:20])/4.0)* 1.0e9


cal19_dead_time = hf['ancillary_data/calibrations/first_photon_bias/gt1l/dead_time'][:]
cal19_strength = hf['ancillary_data/calibrations/first_photon_bias/gt1l/strength'][:][:]
cal19_width = hf['ancillary_data/calibrations/first_photon_bias/gt1l/width'][:][:]
cal19_corr = hf['ancillary_data/calibrations/first_photon_bias/gt1l/ffb_corr'][:][:][:]

hf.close()


##
## Set absolute distance for all photons
##

x_ph = np.zeros(len(ATL03_ph_x))
x_ph = np.float64(x_ph)

for ii in np.arange(0, len(ATL03_ph_index_beg)):
    if (ATL03_seg_ph_count[ii] > 0):
        p0 = ATL03_ph_index_beg[ii] - 1
        p1 = p0 + ATL03_seg_ph_count[ii]
        x_ph[p0:p1] = ATL03_ph_x[p0:p1] + ATL03_seg_dist_x[ii]




print('dist_x')
print(np.min(x_ph), np.max(x_ph))

print('delta time')
print(np.min(ATL03_ph_delta_time), np.max(ATL03_ph_delta_time))

In [ ]:
#
#
###################Read WF tables###################################
# 
# 
# 
print('')
print('Reading wave form table...')
print(wf_table_file1)
print(wf_table_file2)
print('=====================================')

#
# Open file
#
hf = h5py.File(wf_table_file1, 'r')

#
# Read waveform table
#
wf_table1 = hf['wf_table'][:][:][:]

#
# Read ancillary axes arrays
#
wf_bins1 = hf['binz'][:]
wf_sd1 = hf['sdz'][:]
wf_mn1 = hf['mnz'][:]

hf.close()

#
# Open file
#
hf = h5py.File(wf_table_file2, 'r')

#
# Read waveform table
#
wf_table2 = hf['wf_table'][:][:][:]

#
# Read ancillary axes arrays
#
wf_bins2 = hf['binz'][:]
wf_sd2 = hf['sdz'][:]
wf_mn2 = hf['mnz'][:]

hf.close()
print(wf_table1.shape)
print(wf_table2.shape)


#
# Set wf_table to use based on TEP_used from reading ATL07
# (This data can also be found on ATL03)
#

if (tep_used == 1):
    wf_table = wf_table1
    wf_bins = wf_bins1
    wf_sd = wf_sd1
    wf_mn = wf_mn1

if (tep_used == 2):
    wf_table = wf_table2
    wf_bins = wf_bins2
    wf_sd = wf_sd2
    wf_mn = wf_mn2

#
# Set histogram bins
# (lb/ub_bin : CENTER of lowest/highest bin)
#
bin_size = 0.025
lb_bin = -2.0
ub_bin = 3.5

bins_center = np.arange(lb_bin, ub_bin+bin_size, bin_size)
bins_edges = np.arange(lb_bin - bin_size*0.5, ub_bin + bin_size, bin_size)

In [ ]:
###
### Set indicies
###

##
## By default this routine processes 1km of data starting at the mean ATL03 along-track distance
##

##
## The following cell can be modified to select a range of surface segments to process
## Examples include a time range, or potential erroneous segments.
## Add more parameters to the ATL07 read section for further inspection.
##


In [ ]:
x0 = np.nanmean(x_ph[100:])
x1 = x0 + 100.0

process_heights_ndx = np.nonzero(np.logical_and(ATL07_dist_x > x0, ATL07_dist_x < x1) )[0]

print('along-track distance range for processing:')
print(x0,':',x1)

print('N segments:')
print(len(process_heights_ndx))

print('N valid segments:')
print(np.count_nonzero(ATL07_h_corr[process_heights_ndx]<1e38))


In [ ]:
##
## Call fine_track for each of the segments
##

#
# Set up arrays for storing results
#
h_surf_hold = []
w_gauss_hold = []
rms_hold = []
quality_flag_hold = []

#
# Loop through requested segments and compute heights
#
# for i_test in np.arange(int(atl07_i0), int(atl07_i0)+1):
for i_test in process_heights_ndx:

    # print(i_test)
    # print(ATL07_dist_x[i_test])
    # print(ATL07_seg_length[i_test])
#
# Select ATL07 segment
# 
# Add padding by adding 10 percent of segment length to either side
#
    ph_x_0 = ATL07_dist_x[i_test] - (ATL07_seg_length[i_test] + ATL07_seg_length[i_test]*0.10)/2.0
    ph_x_1 = ATL07_dist_x[i_test] + (ATL07_seg_length[i_test] + ATL07_seg_length[i_test]*0.10)/2.0
#
# Collect index of selected photons
# --Photons within distance window and lb/ub_win height
#
    photons_loc = np.where( (x_ph[:] > ph_x_0) & (x_ph[:] < ph_x_1) & ( ATL03_ph_height[:] > (ATL07_coarse_mn[i_test]+lb_bin) ) & ( ATL03_ph_height[:] < (ATL07_coarse_mn[i_test]+ub_bin)) )

#
# Count shots and photons
#
    running_pulse = ((ATL03_mframe[photons_loc] - ATL03_mframe[photons_loc][0]))*200+ATL03_pulse[photons_loc]
    n_shots = np.max(running_pulse) - np.min(running_pulse) + 1
    n_photons = ATL03_ph_delta_time[photons_loc].size
    # print(n_shots,n_photons)

    # print('call finetrack')
    h_surf_temp, w_temp, fpb_corr_m, h_fit_qual_flag, error_surface, \
    hist_norm, bins_trim, wf_table_fit, qtr_h, h_rms, n_photons_trim, \
    hist_full_out, hist_trim3_out, wf_table_trim, wf_bins_trim_, mask = \
    fine_track(0, ATL03_ph_height[photons_loc], ATL07_coarse_mn[i_test], \
    n_photons, n_shots, bin_size, lb_bin, ub_bin, \
    wf_table, wf_bins, wf_sd, wf_mn, \
    cal19_corr, cal19_width, cal19_strength, cal19_dead_time, dead_time_s)


    # print('DONE finetrack')
    # print(h_rms, qtr_h)
    h_surf_hold.append(h_surf_temp)
    w_gauss_hold.append(w_temp)
    rms_hold.append(h_rms)
    quality_flag_hold.append(h_fit_qual_flag)

    
    if (print_debug == 'y'):
        
        output_file = output_path + 'debug_' + str(i_test).zfill(6)
        # plot_finetrack_seg(output_file, ATL03_ph_delta_time[photons_loc], ATL03_ph_height[photons_loc], hist_norm, bins_trim, wf_fit, error_surface, qtr_h)
        plot_finetrack_seg(output_file, x_ph[photons_loc], ATL03_ph_height[photons_loc], hist_norm, bins_trim, wf_table_fit, error_surface, qtr_h,ATL07_quality_flag[i_test],h_fit_qual_flag, ATL07_h_uncorr[i_test], h_surf_temp, ATL07_dist_x[i_test])




In [ ]:
#
# Hide invalid data from plotting
#
ATL07_h_uncorr[ATL07_h_uncorr>1e38] = np.nan
ATL07_w_gauss[ATL07_w_gauss>1e38] = np.nan
ATL07_rms[ATL07_rms>1e38] = np.nan

In [ ]:
##
## Set projection for groundtracks
##

if region == 1:

  clat = 90
  clon = -45
  ts_lat = 70

  inProj = Proj("+init=EPSG:4326")
  outProj = Proj("+init=EPSG:3411")

  proj = ccrs.Stereographic(central_longitude=clon, central_latitude=clat,
                          globe=ccrs.Globe(semimajor_axis=6378273, semiminor_axis=6356889.449),
                          true_scale_latitude=ts_lat)

elif region ==2:

  clat = -90
  clon = 0
  ts_lat = -70
    
  inProj = Proj("+init=EPSG:4326")
  outProj = Proj("+init=EPSG:3412")

  proj = ccrs.Stereographic(central_longitude=clon, central_latitude=clat,
                            globe=ccrs.Globe(semimajor_axis=6378273, semiminor_axis=6356889.449),
                            true_scale_latitude=ts_lat)

x_ground_track, y_ground_track = transform(inProj, outProj, ATL07_lon, ATL07_lat)


In [ ]:
fig = plt.figure(figsize=(10, 12))
gs = GridSpec(nrows=3, ncols=2, figure=fig)

ax0 = fig.add_subplot(gs[0,:], projection=proj)
ax0.title.set_text('ATL07 Ground Track')
ax0.plot(x_ground_track, y_ground_track, '.', c='orange', label= 'ATL07 Heights')
ax0.plot(x_ground_track[process_heights_ndx], y_ground_track[process_heights_ndx], '.', c='green', label= 'Python Heights')
ax0.add_feature(ft.LAND, zorder=100, edgecolor='k', facecolor='none')
ax0.legend()

ax1 = fig.add_subplot(gs[1,0])
ax1.title.set_text('ATL07 vs Python (uncorrected heights)')
ax1.scatter(ATL07_dist_x[process_heights_ndx], ATL07_h_uncorr[process_heights_ndx], alpha=0.7, label='ATL07 height', c='orange')
ax1.scatter(ATL07_dist_x[process_heights_ndx], h_surf_hold, alpha=0.7, label='Python height', c='green')
ax1.legend()

ax2 = fig.add_subplot(gs[1,1])
ax2.title.set_text('ATL07 vs Python (w_gauss)')
ax2.scatter(ATL07_dist_x[process_heights_ndx], ATL07_w_gauss[process_heights_ndx], alpha=0.7, label='ATL07 w_gauss', c='orange')
ax2.scatter(ATL07_dist_x[process_heights_ndx], w_gauss_hold, alpha=0.7, label='Python w_gauss', c='green')
ax2.legend()


ax3 = fig.add_subplot(gs[2,0])
ax3.title.set_text('ATL07 vs Python (RMS)')
ax3.scatter(ATL07_dist_x[process_heights_ndx], ATL07_rms[process_heights_ndx], alpha=0.7, label='ATL07 RMS', c='orange')
ax3.scatter(ATL07_dist_x[process_heights_ndx], rms_hold, alpha=0.7, label='Python RMS', c='green')
ax3.legend()

ax4 = fig.add_subplot(gs[2,1])
ax4.title.set_text('ATL07 vs Python (quality_flag)')
ax4.scatter(ATL07_dist_x[process_heights_ndx], ATL07_quality_flag[process_heights_ndx], alpha=0.7, label='ATL07 quality_flag', c='orange')
ax4.scatter(ATL07_dist_x[process_heights_ndx], quality_flag_hold, alpha=0.7, label='Python quality_flag', c='green')
ax4.legend()

output_file = output_path + 'Summary.png'
fig.savefig(output_file, dpi=fig.dpi)

